# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,damghan,36.1683,54.3480,17.39,44,12,1.33,IR,1696992243
1,1,constantia,44.1833,28.6500,6.50,53,6,3.59,RO,1696992243
2,2,xincheng,33.6333,115.1833,21.91,44,27,2.74,CN,1696992243
3,3,port lincoln,-34.7333,135.8667,26.51,25,89,13.44,AU,1696992244
4,4,ponta delgada,37.7333,-25.6667,22.40,83,20,5.14,PT,1696992244


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points(
         "Lng",
         "Lat",
         geo=True,
         tiles="OSM",
         color="City",
         size="Humidity",title="City Data Map"

)
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values

narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
                                    & (city_data_df["Wind Speed"] < 4.5) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()

# Display sample data
narrowed_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,saint-pierre,-21.3393,55.4781,21.82,56,0,2.06,RE,1696992257
57,57,kourou,5.1552,-52.6478,26.03,83,0,3.00,GF,1696992257
62,62,canico,32.6333,-16.8500,25.62,48,0,1.03,PT,1696992258
135,135,stephenville,32.2207,-98.2023,22.36,63,0,3.60,US,1696992278
242,242,remire-montjoly,4.9167,-52.2667,26.02,89,0,0.51,GF,1696992304
292,292,battagram,34.6772,73.0233,22.16,27,0,1.56,PK,1696992317
312,312,salalah,17.0151,54.0924,25.05,78,0,1.54,OM,1696992323
319,319,jashpurnagar,22.9000,84.1500,25.30,60,0,1.35,IN,1696992325
342,342,kiama,-34.6833,150.8667,25.21,65,0,4.02,AU,1696992332
358,358,las palmas,28.4204,-14.0131,22.44,50,0,3.60,ES,1696992336


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,saint-pierre,RE,-21.3393,55.4781,56,
57,kourou,GF,5.1552,-52.6478,83,
62,canico,PT,32.6333,-16.8500,48,
135,stephenville,US,32.2207,-98.2023,63,
242,remire-montjoly,GF,4.9167,-52.2667,89,
292,battagram,PK,34.6772,73.0233,27,
312,salalah,OM,17.0151,54.0924,78,
319,jashpurnagar,IN,22.9000,84.1500,60,
342,kiama,AU,-34.6833,150.8667,65,
358,las palmas,ES,28.4204,-14.0131,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params={"categories":"accommodation.hotel",
       "apiKey": geoapify_key,  
          "limit":20}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
   
    latitude  = row["Lat"]
    longitude= row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
  # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
kourou - nearest hotel: SCI Horizon
canico - nearest hotel: Dom Pedro
stephenville - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
battagram - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
jashpurnagar - nearest hotel: No hotel found
kiama - nearest hotel: Nova Kiama
las palmas - nearest hotel: Era de La Corte
smara - nearest hotel: Hôtel Amine فندق الأمين
shreveport - nearest hotel: Sam's Town Hotel and Casino Shreveport
mingora - nearest hotel: Chinar Hotel
maun - nearest hotel: Center Lodge Conference Center
mana - nearest hotel: Le Samana
port arthur - nearest hotel: No hotel found
seminole - nearest hotel: Best Western Plus Sanford
formosa - nearest hotel: Hotel Hostal del Rey
tindouf - nearest hotel: محمد بوسبي
diboll - nearest hotel: No hotel found
la junta - nearest hotel: Travel Inn of La Junta


,City,Country,Lat,Lng,Humidity,Hotel Name
55,saint-pierre,RE,-21.3393,55.4781,56,Tropic Hotel
57,kourou,GF,5.1552,-52.6478,83,SCI Horizon
62,canico,PT,32.6333,-16.8500,48,Dom Pedro
135,stephenville,US,32.2207,-98.2023,63,No hotel found
242,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
292,battagram,PK,34.6772,73.0233,27,No hotel found
312,salalah,OM,17.0151,54.0924,78,Muscat International Hotel
319,jashpurnagar,IN,22.9000,84.1500,60,No hotel found
342,kiama,AU,-34.6833,150.8667,65,Nova Kiama
358,las palmas,ES,28.4204,-14.0131,50,Era de La Corte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_plot= hotel_df.hvplot.points(
         "Lng",
         "Lat",
         geo=True,
         tiles="OSM",
         color="City",
         size="Humidity",
         hover_cols=["Hotel Name","Country"],title="Hotel Data Map")


hotel_plot
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)